# Extraction of SWR from CA1 recordings

Restarting from LFPwake0 and LFPwakeremoved.

LFPwakeremoved will be used to determined signal variance for threshold adjustement. 

LFPwake0 will be used for time determination. 

## Load LFP and packages

In [37]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy import fftpack
import pandas as pd
from pathlib import Path
import os
from ipyfilechooser import FileChooser

%matplotlib widget

from ephyviewer import mkQApp, MainViewer, TraceViewer, EventList, InMemoryEventSource
from ephyviewer import AnalogSignalSourceWithScatter
import ephyviewer

In [5]:
dpath = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording/GreenLinesOK/session5/OpenEphys"
try:
    %store -r dpath
except:
    print("data path not in strore")
    dpath = os.path.expanduser("~")

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>OpenEphys Folder</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

FileChooser(path='/Users/mb', filename='', title='<b>OpenEphys Folder</b>', show_hidden=False, select_desc='Se…

Stored 'dpath' (str)


In [7]:
suffix=''#'_AB'
sep = -5
animalIDPos = -3
dirPathComponents = os.path.normpath(dpath).split(os.sep)
rootPath = os.path.join('/',*dirPathComponents[:sep])
folder_base = os.path.join(*dirPathComponents[sep:])
mice = "GreenLinesOK" #dirPathComponents[animalIDPos]
os.chdir(rootPath)
print(rootPath)
print(folder_base)
print(mice)

/Users/mb
Documents/Syntuitio/AudreyHay/PlanB/session5
GreenLinesOK


In [9]:
filename = os.path.join(folder_base,f'LFPwake0{suffix}.npy')
filename3 = os.path.join(folder_base,f'LFPwakeremoved{suffix}.npy')
filename2 = os.path.join(folder_base,'RawDataChannelExtractedDS.npy')
EMGbooleaninput = os.path.join(folder_base,f'EMGframeBoolean{suffix}.pkl')
Channels = os.path.join(folder_base,f'LFPChannels_perMice.xlsx')

EMGboolean = pd.read_pickle(EMGbooleaninput)
LFPwakeremoved = np.load(filename3, mmap_mode= 'r')
All = np.load(filename2, mmap_mode= 'r')

allchannels = pd.read_excel(Channels)
PFCch1=int(allchannels[mice][0].split(',')[0])
PFCch2=int(allchannels[mice][0].split(',')[1])
CA1ch1=int(allchannels[mice][2].split(',')[0])
CA1ch2=int(allchannels[mice][2].split(',')[1])
#CA1ch3=int(allchannels[mice][2].split(',')[2])

PFC  =  All[:, PFCch1]-All[:, PFCch2] 
CA1  =  All[:, CA1ch1]-All[:, CA1ch2] 
CA1wakeremoved = LFPwakeremoved[:,CA1ch1]-LFPwakeremoved[:,CA1ch2] 

# Band pass filter
        SWR: 120-200 Hz

In [10]:
# Filtre parameter:
f_lowcut = 120.
f_hicut = 200.
fs = 1000
nyq = 0.5 * fs
N = 6                 # Filtre order
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction

# Filtering:
b, a = signal.butter(N, Wn, 'band')
filt_CA1 = signal.filtfilt(b, a, CA1)
filt_CA1wakeremoved = signal.filtfilt(b, a, CA1wakeremoved)

# Plot
times = np.arange(0, CA1.size/fs, 1./fs)
#timesmin = np.arange(0, CA1.size/fs/60, 1./fs/60)
#fig, ax = plt.subplots()
#ax.plot(timesmin, filt_CA1)

## Continuous Wavelet Transform and projection calculation

First on signal with no wake time to determine sd of signal

In [11]:
# Parameter and computation of CWT
w = 10.
freq = np.linspace(120, 200, 80)
widths = w*fs / (2*freq*np.pi)
CA1NWcwt = signal.cwt(filt_CA1wakeremoved, signal.morlet2, widths, w=w)

# Projection calculation
absCA1NWcwt = np.absolute(CA1NWcwt)
proj_CA1NWcwt = np.sum(absCA1NWcwt, axis = 0)/80
sdproj_CA1cwt = np.std(proj_CA1NWcwt)
sd3proj_CA1cwt = sdproj_CA1cwt*3
sd10proj_CA1cwt = sdproj_CA1cwt*10
sd8proj_CA1cwt = sdproj_CA1cwt*8
sd7proj_CA1cwt = sdproj_CA1cwt*7
sd05proj_CA1cwt = sdproj_CA1cwt*0.5

/var/folders/2_/fpwjwx512255f113wntjtndm0000gn/T/ipykernel_70990/3157878907.py:5: DeprecationWarning: scipy.signal.cwt is deprecated in SciPy 1.12 and will be removed
in SciPy 1.15. We recommend using PyWavelets instead.

  CA1NWcwt = signal.cwt(filt_CA1wakeremoved, signal.morlet2, widths, w=w)


Second on the signal for which wake times have been zeroed

In [12]:
# Conservative boolean filtering of CA1 filtered signal
BooleanCons = EMGboolean['BooleanConservative']
fCA1wake0C = filt_CA1.copy()
fCA1wake0C[BooleanCons] = 0
CA1wake0C = CA1.copy()
CA1wake0C[BooleanCons] = 0
# Liberal boolean filtering of CA1 filtered signal
BooleanLib = EMGboolean['BooleanLiberal']
fCA1wake0L = filt_CA1.copy()
fCA1wake0L[BooleanLib] = 0
CA1wake0L = CA1.copy()
CA1wake0L[BooleanLib] = 0

# Computation of CWT
CA1cwtWake0cons = signal.cwt(fCA1wake0C, signal.morlet2, widths, w=w)
CA1cwtWake0lib = signal.cwt(fCA1wake0L, signal.morlet2, widths, w=w)

# Projection calculation
absCA1W0Ccwt = np.absolute(CA1cwtWake0cons)
proj_CA1W0Ccwt = np.sum(absCA1W0Ccwt, axis = 0)/80
absCA1W0Lcwt = np.absolute(CA1cwtWake0lib)
proj_CA1W0Lcwt = np.sum(absCA1W0Lcwt, axis = 0)/80

combined = np.stack([CA1, filt_CA1, proj_CA1W0Ccwt, proj_CA1W0Lcwt], axis = 1)

sample_rate = 1000.
t_start = 0.

/var/folders/2_/fpwjwx512255f113wntjtndm0000gn/T/ipykernel_70990/2628973646.py:15: DeprecationWarning: scipy.signal.cwt is deprecated in SciPy 1.12 and will be removed
in SciPy 1.15. We recommend using PyWavelets instead.

  CA1cwtWake0cons = signal.cwt(fCA1wake0C, signal.morlet2, widths, w=w)
/var/folders/2_/fpwjwx512255f113wntjtndm0000gn/T/ipykernel_70990/2628973646.py:16: DeprecationWarning: scipy.signal.cwt is deprecated in SciPy 1.12 and will be removed
in SciPy 1.15. We recommend using PyWavelets instead.

  CA1cwtWake0lib = signal.cwt(fCA1wake0L, signal.morlet2, widths, w=w)


## Extracting SWRs and determining main properties 

First extraction of SWR peaks, initiation, end and width

In [13]:
from scipy.signal import find_peaks
from scipy.signal import chirp, find_peaks, peak_widths

# 10 sd threshold
peaks, properties = find_peaks(proj_CA1W0Lcwt, prominence=1, width=20, height=sd8proj_CA1cwt) #AB detection with 8*SD // Audrey's detection=3*SD
properties["prominences"], properties["widths"]

# SWR boundaries taken at 70% from peak of intensity. This means that the SWRs with small amplitude will be longer than the big ones.
results_width = peak_widths(proj_CA1W0Lcwt, peaks, rel_height=0.7)

# Organise results in numpy array
peaks2 = peaks.reshape(len(peaks),1)
npresults_width = np.array(results_width).reshape(4,-1)
SWR_prop = np.append(peaks2, results_width).reshape(5,len(peaks2)).round()

Display subset

Second extraction of main frequency and power 

In [29]:
projMaxP_cwtmg = np.max(CA1cwtWake0lib, axis = 0)
projMaxF_cwtmg = np.argmax(CA1cwtWake0lib, axis = 0) + 120
projMaxP_cwtmg.shape

nb_SWR = len(peaks)
data = np.zeros((nb_SWR,4))

for tt in np.arange(nb_SWR):
    SWR_start = int(SWR_prop[3,tt])
    SWR_stop = int(SWR_prop[4,tt])
    SWR_MaxP = projMaxP_cwtmg[SWR_start:SWR_stop]
    SWR_MaxF = projMaxF_cwtmg[SWR_start:SWR_stop]
    data[tt, 0] = max(SWR_MaxF).round()
    data[tt, 1] = max(SWR_MaxP).round()
    data[tt, 2] = round(sum(SWR_MaxF)/len(SWR_MaxF))
    data[tt, 3] = round(sum(SWR_MaxP)/len(SWR_MaxP))

param_SWR = pd.DataFrame(data, columns = ['Max freq', 'Max int', 'Avg freq', 'Avg int'])
tSWR_prop = SWR_prop.transpose()
pd_prop_SWR = pd.DataFrame(tSWR_prop, columns = ['peak time', 'Duration', 'peak amp', 'start time', 'end time'])
pd_tokeep = pd.DataFrame(np.ones(nb_SWR).astype(bool), columns = ['toKeep'])
All_SWR = pd.concat([pd_tokeep,pd_prop_SWR, param_SWR], axis=1)

SWR_peak = peaks
SWR_start = SWR_prop[3,:].astype(int)
SWR_end = SWR_prop[4,:].astype(int)

/var/folders/2_/fpwjwx512255f113wntjtndm0000gn/T/ipykernel_70990/2193912474.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
  data[tt, 1] = max(SWR_MaxP).round()
/var/folders/2_/fpwjwx512255f113wntjtndm0000gn/T/ipykernel_70990/2193912474.py:16: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  data[tt, 3] = round(sum(SWR_MaxP)/len(SWR_MaxP))
/var/folders/2_/fpwjwx512255f113wntjtndm0000gn/T/ipykernel_70990/2193912474.py:16: ComplexWarning: Casting complex values to real discards the imaginary part
  data[tt, 3] = round(sum(SWR_MaxP)/len(SWR_MaxP))


### Store the results in All_SWR_prop pd dataframe and save as pkl/csv for post processing.

End of Notebook. 

In [30]:
filename2 = os.path.join(folder_base,f'SWRproperties{suffix}.pkl')
filename3 = os.path.join(folder_base,f'SWRproperties{suffix}.csv')
All_SWR.to_pickle(filename2)
All_SWR.to_csv(filename3, sep = ',')

#quoted cause otherwise error plotting ch2 & ch3 in the next display
""" 
combined = np.stack([fCA1wake0L, proj_CA1W0Lcwt], axis = 1)
filenameC = folder_base / f'SignalCA1.npy'
np.save(filenameC, combined)
"""
# if done and no intention to display for assessment
#%reset
#plt.close('all')

" \ncombined = np.stack([fCA1wake0L, proj_CA1W0Lcwt], axis = 1)\nfilenameC = folder_base / f'SignalCA1.npy'\nnp.save(filenameC, combined)\n"

Load previous SWR detection (only if wanted)

In [12]:
"""
OldSWSdetection=folder_base / f'SWRproperties.csv'
SWR_prop = pd.read_csv(OldSWSdetection, index_col=0)            
pd_prop_SWR=All_SWR.iloc[:, :5]
param_SWR=All_SWR.iloc[:, 5:9]
SWR_prop=SWR_prop.values
SWR_peak = np.transpose(SWR_prop[:,0].astype(int))
SWR_start = np.transpose(SWR_prop[:,3].astype(int))
SWR_end = np.transpose(SWR_prop[:,4].astype(int))
"""

"\nOldSWSdetection=folder_base / f'SWRproperties.csv'\nSWR_prop = pd.read_csv(OldSWSdetection, index_col=0)            \npd_prop_SWR=All_SWR.iloc[:, :5]\nparam_SWR=All_SWR.iloc[:, 5:9]\nSWR_prop=SWR_prop.values\nSWR_peak = np.transpose(SWR_prop[:,0].astype(int))\nSWR_start = np.transpose(SWR_prop[:,3].astype(int))\nSWR_end = np.transpose(SWR_prop[:,4].astype(int))\n"

### Display

ephys viewer to check SWR detection

In [73]:
%gui qt
app = mkQApp()

#Create one data source with 3 event channel
all_events = []
conditions = ['All','Good','Bad']
for c,cond in enumerate(conditions):
    match cond:
        case 'All':
            selection = "All_SWR['toKeep'] | ~All_SWR['toKeep']"
        case 'Good':
            selection = "All_SWR['toKeep']"
        case 'Bad':
            selection = "~All_SWR['toKeep']"
    ev_times = All_SWR.loc[pd.eval(selection),'peak time'].values
    ev_labels = [f'SWR {i}'for i in All_SWR[pd.eval(selection)].index]
    all_events.append({ 'time':ev_times, 'label':ev_labels, 'name': conditions[c] })
source_ev = InMemoryEventSource(all_events=all_events)



sample_rate = 1000.
t_start = 0.

#create 2 familly scatters from theses 2 indexes
scatter_indexes = {0: SWR_peak, 1: SWR_start, 2: SWR_end}
#and asign them to some channels each
scatter_channels = {0: [1, 2], 1: [0, 1], 2: [0, 1]}
source = AnalogSignalSourceWithScatter(combined, sample_rate, t_start, scatter_indexes, scatter_channels)

#Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

#create a viewer for signal with TraceViewer
#connected to the signal source
view1 = TraceViewer(source=source)

#Parameters can be set in script
#view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
#And also parameters for each channel
view1.by_channel_params['ch0', 'color'] = '#ffffff'
view1.by_channel_params['ch1', 'color'] = '#0055ff'
view1.by_channel_params['ch2', 'color'] = '#ff5500'
view1.by_channel_params['ch3', 'color'] = '#ffffff'

view1.by_channel_params['ch0', 'gain'] = 0.001
view1.by_channel_params['ch1', 'gain'] = 0.005
view1.by_channel_params['ch2', 'gain'] = 0.005
view1.by_channel_params['ch3', 'gain'] = 0.005


view1.by_channel_params['ch0', 'offset'] = 6
view1.by_channel_params['ch1', 'offset'] = -1
view1.by_channel_params['ch2', 'offset'] = -1
view1.by_channel_params['ch3', 'offset'] = -1


view2 = EventList(source=source_ev, name='event')


#put this viewer in the main window
win.add_view(view1)
win.add_view(view2, location='bottom',  orientation='horizontal')

#Run
win.show()
app.exec_()

    toKeep  peak time  Duration  peak amp  start time   end time  Max freq  \
0     True     23.064      78.0      78.0     23019.0    23097.0     199.0   
1     True     63.451     168.0      83.0     63331.0    63498.0     199.0   
2     True     67.193      38.0      92.0     67173.0    67211.0     195.0   
3     True     67.348      53.0     113.0     67328.0    67381.0     199.0   
4     True     71.428      41.0      99.0     71405.0    71446.0     195.0   
5    False     86.350      74.0      85.0     86307.0    86381.0     196.0   
6     True     88.724      84.0      79.0     88696.0    88780.0     199.0   
7     True    105.038      53.0      82.0    105019.0   105072.0     196.0   
8     True    160.836      60.0      82.0    160817.0   160877.0     198.0   
9     True    184.296      61.0      84.0    184262.0   184323.0     199.0   
10    True    211.661     132.0      87.0    211625.0   211757.0     199.0   
11    True    227.443      59.0      80.0    227423.0   227482.0

/var/folders/2_/fpwjwx512255f113wntjtndm0000gn/T/ipykernel_70990/1082842728.py:70: DeprecationWarning: 'exec_' will be removed in the future. Use 'exec' instead.
  app.exec_()


refresh duration for  0.00022912025451660156 s
refresh duration for  0.00034880638122558594 s
refresh duration for  0.0007250308990478516 s
refresh duration for  0.00028586387634277344 s


qt.pointer.dispatch: skipping QEventPoint(id=0 ts=0 pos=0,0 scn=530.838,339.571 gbl=530.838,339.571 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-530.838,-339.571 last=-530.838,-339.571 Δ 530.838,339.571) : no target window
qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=593.477,325.349 gbl=593.477,325.349 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-593.477,-325.349 last=-593.477,-325.349 Δ 593.477,325.349) : no target window


refresh duration for  0.0003910064697265625 s
refresh duration for  0.0003910064697265625 s


qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=901.313,430.525 gbl=901.313,430.525 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-901.313,-430.525 last=-901.313,-430.525 Δ 901.313,430.525) : no target window


save_all_settings


0

Quick plot of average and max intensity (can be done with freq as well) paired.